라이브러리 불러오기

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

데이터 불러오기

In [2]:
# 착지 파일 업로드
landing_df = pd.read_csv('착지.csv')

# 주문 파일 업로드
order_mon_df = pd.read_csv('주문_월요일.csv')
order_tue_df = pd.read_csv('주문_화요일.csv')
order_wed_df = pd.read_csv('주문_수요일.csv')
order_thu_df = pd.read_csv('주문_목요일.csv')
order_fri_df = pd.read_csv('주문_금요일.csv')
order_sat_df = pd.read_csv('주문_토요일.csv')

# OD Matrix 파일 업로드
OD_csv = pd.read_csv('updated_OD_Matrix_corrected 3.csv')

# 센터 파일 업로드
center = pd.read_csv('센터.csv')

# 차량 파일 업로드 
vehicle = pd.read_csv('차량정보.csv')

In [3]:
# 통합 주문 데이터프레임
order_dfs = [order_mon_df, order_tue_df, order_wed_df, order_thu_df, order_fri_df, order_sat_df ]
order_df = pd.concat(order_dfs, ignore_index=True)

착지 테이블과 주문 테이블 합치기

In [4]:
# ID와 Stop_ID를 기준으로 병합
integration_mon_df = pd.merge(landing_df[:], order_mon_df[:], left_on='ID', right_on='Stop_ID')
integration_tue_df = pd.merge(landing_df[:], order_tue_df[:], left_on='ID', right_on='Stop_ID')
integration_wed_df = pd.merge(landing_df[:], order_wed_df[:], left_on='ID', right_on='Stop_ID')
integration_thu_df = pd.merge(landing_df[:], order_thu_df[:], left_on='ID', right_on='Stop_ID')
integration_fri_df = pd.merge(landing_df[:], order_fri_df[:], left_on='ID', right_on='Stop_ID')
integration_sat_df = pd.merge(landing_df[:], order_sat_df[:], left_on='ID', right_on='Stop_ID')

# 불필요한 열 제거 (Y_y, X_y만 제거, Stop_ID는 Order_ID와 함께 유지)
integration_mon_df = integration_mon_df.drop(columns=['Y_y', 'X_y'])
integration_tue_df = integration_tue_df.drop(columns=['Y_y', 'X_y'])
integration_wed_df = integration_wed_df.drop(columns=['Y_y', 'X_y'])
integration_thu_df = integration_thu_df.drop(columns=['Y_y', 'X_y'])
integration_fri_df = integration_fri_df.drop(columns=['Y_y', 'X_y'])
integration_sat_df = integration_sat_df.drop(columns=['Y_y', 'X_y'])

# 열 이름 변경
integration_mon_df = integration_mon_df.rename(columns={'Y_x': 'Y', 'X_x': 'X'})
integration_tue_df = integration_tue_df.rename(columns={'Y_x': 'Y', 'X_x': 'X'})
integration_wed_df = integration_wed_df.rename(columns={'Y_x': 'Y', 'X_x': 'X'})
integration_thu_df = integration_thu_df.rename(columns={'Y_x': 'Y', 'X_x': 'X'})
integration_fri_df = integration_fri_df.rename(columns={'Y_x': 'Y', 'X_x': 'X'})
integration_sat_df = integration_sat_df.rename(columns={'Y_x': 'Y', 'X_x': 'X'})

# 열 순서 재정렬
cols_1 = ['Order_ID', 'ID'] + [col for col in integration_mon_df.columns if col not in ['Order_ID', 'ID']]
cols_2 = ['Order_ID', 'ID'] + [col for col in integration_tue_df.columns if col not in ['Order_ID', 'ID']]
cols_3 = ['Order_ID', 'ID'] + [col for col in integration_wed_df.columns if col not in ['Order_ID', 'ID']]
cols_4 = ['Order_ID', 'ID'] + [col for col in integration_thu_df.columns if col not in ['Order_ID', 'ID']]
cols_5 = ['Order_ID', 'ID'] + [col for col in integration_fri_df.columns if col not in ['Order_ID', 'ID']]
cols_6 = ['Order_ID', 'ID'] + [col for col in integration_sat_df.columns if col not in ['Order_ID', 'ID']]

integration_mon_df = integration_mon_df[cols_1]
integration_tue_df = integration_tue_df[cols_2]
integration_wed_df = integration_wed_df[cols_3]
integration_thu_df = integration_thu_df[cols_4]
integration_fri_df = integration_fri_df[cols_5]
integration_sat_df = integration_sat_df[cols_6]

진입제한이 1000인 테이블, 진입제한이 1000이 넘으면서 무게가 3000을 안 넘는 테이블, 진입제한이 1000을 넘으면서 무게가 3000을 넘는 테이블로 나누기

In [5]:
one_mon_df = integration_mon_df[integration_mon_df['Access_restriction'] == 1000]
one_tue_df = integration_tue_df[integration_tue_df['Access_restriction'] == 1000]
one_wed_df = integration_wed_df[integration_wed_df['Access_restriction'] == 1000]
one_thu_df = integration_thu_df[integration_thu_df['Access_restriction'] == 1000]
one_fri_df = integration_fri_df[integration_fri_df['Access_restriction'] == 1000]
one_sat_df = integration_sat_df[integration_sat_df['Access_restriction'] == 1000]
else_mon_df = integration_mon_df[integration_mon_df['Access_restriction'] != 1000]
else_tue_df = integration_tue_df[integration_tue_df['Access_restriction'] != 1000]
else_wed_df = integration_wed_df[integration_wed_df['Access_restriction'] != 1000]
else_thu_df = integration_thu_df[integration_thu_df['Access_restriction'] != 1000]
else_fri_df = integration_fri_df[integration_fri_df['Access_restriction'] != 1000]
else_sat_df = integration_sat_df[integration_sat_df['Access_restriction'] != 1000]

In [6]:
three_mon_df = else_mon_df[(else_mon_df['weight'] <= 3500) & (else_mon_df['plt'] <= 8)]
three_tue_df = else_tue_df[(else_tue_df['weight'] <= 3500) & (else_tue_df['plt'] <= 8)]
three_wed_df = else_wed_df[(else_wed_df['weight'] <= 3500) & (else_wed_df['plt'] <= 8)]
three_thu_df = else_thu_df[(else_thu_df['weight'] <= 3500) & (else_thu_df['plt'] <= 8)]
three_fri_df = else_fri_df[(else_fri_df['weight'] <= 3500) & (else_fri_df['plt'] <= 8)]
three_sat_df = else_sat_df[(else_sat_df['weight'] <= 3500) & (else_sat_df['plt'] <= 8)]
eleven_mon_df = else_mon_df[(else_mon_df['weight'] > 3500) | (else_mon_df['plt'] > 8)]
eleven_tue_df = else_tue_df[(else_tue_df['weight'] > 3500) | (else_tue_df['plt'] > 8)]
eleven_wed_df = else_wed_df[(else_wed_df['weight'] > 3500) | (else_wed_df['plt'] > 8)]
eleven_thu_df = else_thu_df[(else_thu_df['weight'] > 3500) | (else_thu_df['plt'] > 8)]
eleven_fri_df = else_fri_df[(else_fri_df['weight'] > 3500) | (else_fri_df['plt'] > 8)]
eleven_sat_df = else_sat_df[(else_sat_df['weight'] > 3500) | (else_sat_df['plt'] > 8)]

클러스터링 함수

In [7]:
def clustering(order_df, num_cluster):
    # Step 1: 착지 파일에서 Start_time과 End_time을 기준으로 클러스터링 (4개 클러스터)
    kmeans_time = KMeans(n_clusters=4, random_state=42)
    landing_df['Cluster_time'] = kmeans_time.fit_predict(landing_df[['Start_time', 'End_time']])

    # Step 2: 각 클러스터별로 ID 리스트를 생성
    cluster_id_lists = [[] for _ in range(4)]
    for cluster_num in range(4):
        cluster_id_lists[cluster_num] = landing_df[landing_df['Cluster_time'] == cluster_num]['ID'][:].tolist()

    # Step 3: 주문 파일에서 각 클러스터 리스트의 ID와 Stop_ID를 매칭하여 클러스터별로 데이터 분리 및 클러스터링
    location_cluster_dfs = []
    for cluster_num, id_list in enumerate(cluster_id_lists):
        cluster_time = order_df[order_df['Stop_ID'].isin(id_list)].copy()
        
        # Step 4: 각 클러스터 내에서 X와 Y를 기준으로 다시 클러스터링 (예: num_cluster 개 클러스터)
        kmeans_location = KMeans(n_clusters=num_cluster, random_state=42)
        cluster_time['Cluster_location'] = kmeans_location.fit_predict(cluster_time[['X', 'Y']])
        
        # 각 위치 클러스터별 데이터프레임을 저장할 리스트
        cluster_location_dfs = []
        for inner_cluster_num in range(num_cluster):
            cluster_location_df = cluster_time[cluster_time['Cluster_location'] == inner_cluster_num].copy()
            cluster_location_dfs.append(cluster_location_df)
        
        # 각 시간별 클러스터에 해당하는 위치별 클러스터 데이터프레임을 저장
        location_cluster_dfs.append(cluster_location_dfs)
    
    return location_cluster_dfs

In [8]:
mon_cluster_1 = clustering(one_mon_df, 2)
tue_cluster_1 = clustering(one_tue_df, 2)
wed_cluster_1 = clustering(one_wed_df, 2)
thu_cluster_1 = clustering(one_thu_df, 2)
fri_cluster_1 = clustering(one_fri_df, 2)
sat_cluster_1 = clustering(one_sat_df, 2)
mon_cluster_3 = clustering(three_mon_df, 7)
tue_cluster_3 = clustering(three_tue_df, 7)
wed_cluster_3 = clustering(three_wed_df, 7)
thu_cluster_3 = clustering(three_thu_df, 7)
fri_cluster_3 = clustering(three_fri_df, 7)
sat_cluster_3 = clustering(three_sat_df, 4)
mon_cluster_11 = clustering(eleven_mon_df, 2)
tue_cluster_11 = clustering(eleven_tue_df, 2)
wed_cluster_11 = clustering(eleven_wed_df, 2)
thu_cluster_11 = clustering(eleven_thu_df, 2)
fri_cluster_11 = clustering(eleven_fri_df, 2)
sat_cluster_11 = clustering(eleven_sat_df, 2)

착지별 OD Matrix 함수

In [9]:
def OD_Matrix(Bin):
    # OriginID와 DestinationID 리스트
    origin_ids = Bin[:]
    destination_ids = Bin[:]

    # 리스트에 termianl 추가
    origin_ids.append('Terminal')
    destination_ids.append('Terminal')

    # 데이터를 필터링하여 필요한 OriginID와 DestinationID만 남기기
    filtered_data = OD_csv[(OD_csv['OriginID'].isin(origin_ids)) & OD_csv['DestinationID'].isin(destination_ids)]

    # 피벗 테이블을 사용하여 2차원 OD Matrix 만들기 (Total_Time 기준)
    od_time = filtered_data.pivot(index='OriginID', columns='DestinationID', values='Total_Time')
    od_distance = filtered_data.pivot(index='OriginID', columns='DestinationID', values='Total_Distance')

    return od_distance, od_time

Bin 생성 알고리즘

In [10]:
def sort_and_make_bin(cluster, max_weight, max_plt, MaxCount):
    # 각 Stop_ID에 해당하는 Order_ID 리스트와 총 무게, total_plt를 계산하여 데이터프레임 생성
    stop_order_map = cluster.groupby('Stop_ID')['Order_ID'].apply(list).reset_index()
    stop_order_map['total_weight'] = cluster.groupby('Stop_ID')['weight'].sum().values
    stop_order_map['total_plt'] = cluster.groupby('Stop_ID')['plt'].sum().values

    # total_weight를 기준으로 내림차순 정렬하여 무거운 주문부터 처리
    stop_order_map = stop_order_map.sort_values(by='total_weight', ascending=False).reset_index(drop=True)
    
    grouped_orders = []  # 그룹화된 주문을 저장할 리스트
    grouped_stop_ids = []  # 그룹화된 Stop_ID를 저장할 리스트
    group_weights = []  # 각 그룹의 총 무게 리스트 (이차원 리스트)
    group_plts = []  # 각 그룹의 총 팔레트 수 리스트 (이차원 리스트)
    
    current_group = []   # 현재 그룹의 주문을 저장할 리스트
    current_stop_ids = []  # 현재 그룹의 Stop_ID를 저장할 리스트
    current_weight = 0   # 현재 그룹의 총 무게를 저장할 변수
    current_plt = 0      # 현재 그룹의 총 팔레트 수를 저장할 변수
    not_check_orders = list(cluster['Order_ID'])  # 처리되지 않은 주문 ID 리스트
    
    while not_check_orders:  # not_check_orders 리스트가 비어있지 않은 동안 루프를 계속 실행. 즉, 처리되지 않은 주문이 남아 있는 한 계속 실행
        any_order_processed = False # 루프가 한 번이라도 실행되는지 확인하기 위한 플래그 변수 초기화
        
        for idx, row in stop_order_map.iterrows():  # stop_order_map의 각 행을 순회하며 Stop_ID와 Order_ID를 가져옴
            stop_id = row['Stop_ID']
            orders = row['Order_ID']
            
            # 같은 착지의 주문들은 무조건 같은 그룹에 포함
            group_weight = sum(cluster[cluster['Order_ID'].isin(orders)]['weight']) # orders 리스트에 포함된 주문들의 총 무게를 계산하여 group_weight 변수에 저장
            group_plt = sum(cluster[cluster['Order_ID'].isin(orders)]['plt']) # orders 리스트에 포함된 주문들의 총 팔레트 수를 계산하여 group_plt 변수에 저장
            
            if group_weight > max_weight or group_plt > max_plt:
                # 큰 주문들을 나눠서 처리
                order_details = cluster[cluster['Order_ID'].isin(orders)][['Order_ID', 'weight', 'plt']].values.tolist()
                order_details.sort(key=lambda x: (-x[1], -x[2])) # 무게와 팔레트 수로 내림차순 정렬
                
                temp_group = []
                temp_weight = 0
                temp_plt = 0
                
                for order in order_details:
                    if temp_weight + order[1] > max_weight or temp_plt + order[2] > max_plt:
                        if temp_group:
                            grouped_orders.append([temp_group])
                            grouped_stop_ids.append([stop_id])
                            group_weights.append([temp_weight])
                            group_plts.append([temp_plt])
                            temp_group = []
                            temp_weight = 0
                            temp_plt = 0
                    temp_group.append(order[0])
                    temp_weight += order[1]
                    temp_plt += order[2]
                    if order[0] in not_check_orders:
                        not_check_orders.remove(order[0])
                
                if temp_group:
                    grouped_orders.append([temp_group])
                    grouped_stop_ids.append([stop_id])
                    group_weights.append([temp_weight])
                    group_plts.append([temp_plt])
                
                any_order_processed = True
                
            else:
                if (current_weight + group_weight <= max_weight and 
                    current_plt + group_plt <= max_plt and 
                    len(current_stop_ids) + 1 <= MaxCount): # 현재 그룹에 새로운 주문들을 추가했을 때, 최대 무게, 최대 팔레트 수 및 최대 착지 수를 초과하지 않는지 확인
                    current_group.append(orders) # orders 리스트의 주문들을 current_group에 추가
                    current_weight += group_weight 
                    current_plt += group_plt
                    if stop_id not in current_stop_ids: # 현재 그룹에 stop_id가 포함되어 있지 않다면 current_stop_ids에 추가
                        current_stop_ids.append(stop_id)
                    for order in orders:
                        if order in not_check_orders: # 주문이 not_check_orders에 있으면
                            not_check_orders.remove(order) # not_check_orders에서 주문을 제거
                    any_order_processed = True
            
            if current_weight >= max_weight or current_plt >= max_plt or len(current_stop_ids) >= MaxCount: 
                # 현재 그룹의 총 무게가 최대 무게에 도달하거나, 총 팔레트 수가 최대 팔레트 수에 도달했거나, 착지 수가 최대 착지 수에 도달했는지 확인
                break # 이 조건이 참이면, 더 이상 현재 그룹에 주문을 추가하지 않기 위해 반복문을 종료
        
        # 현재 그룹을 저장하고 새로운 그룹 시작
        if any_order_processed: # 이번 루프에서 주문이 추가되었는지 확인해서 만약 하나라도 추가된 경우, 현재 그룹을 저장하고 새로운 그룹을 시작
            stop_weights = [sum(cluster[(cluster['Stop_ID'] == sid) & (cluster['Order_ID'].isin(sum(current_group, [])))]['weight']) for sid in current_stop_ids]
            stop_plts = [sum(cluster[(cluster['Stop_ID'] == sid) & (cluster['Order_ID'].isin(sum(current_group, [])))]['plt']) for sid in current_stop_ids]
            grouped_orders.append(current_group)
            grouped_stop_ids.append(current_stop_ids)
            group_weights.append(stop_weights)
            group_plts.append(stop_plts)
            current_group = [] # current_group을 초기화하여 새로운 그룹을 시작할 준비
            current_stop_ids = [] # current_stop_ids를 초기화하여 새로운 그룹을 시작할 준비
            current_weight = 0 # current_weight 초기화
            current_plt = 0 # current_plt 초기화
            
            # stop_order_map에서 이미 처리된 주문을 제거
            # apply 함수로 각 Order_ID 리스트가 not_check_orders에 없는 경우를 필터링
            stop_order_map = stop_order_map[~stop_order_map['Order_ID'].apply(lambda x: all(order not in not_check_orders for order in x))]
            stop_order_map.reset_index(drop=True, inplace=True) # stop_order_map의 인덱스를 재설정하고, drop=True로 기존 인덱스를 제거
        
        # 무한 루프 방지: 주문이 제거되지 않은 경우 루프를 탈출
        if not any_order_processed:
            break

    # 남은 그룹 추가
    if current_group: # 현재 그룹에 주문이 남아 있는 경우, 이 그룹을 저장
        stop_weights = [sum(cluster[(cluster['Stop_ID'] == sid) & (cluster['Order_ID'].isin(sum(current_group, [])))]['weight']) for sid in current_stop_ids]
        stop_plts = [sum(cluster[(cluster['Stop_ID'] == sid) & (cluster['Order_ID'].isin(sum(current_group, [])))]['plt']) for sid in current_stop_ids]
        grouped_orders.append(current_group)
        grouped_stop_ids.append(current_stop_ids)
        group_weights.append(stop_weights)
        group_plts.append(stop_plts)

    # grouped_orders를 3차원 리스트로 변환 (Stop_ID 기준으로 나누기)
    final_grouped_orders = []
    for group in grouped_orders:
        temp_group = []
        for stop_id_list in group:
            temp_group.append(stop_id_list)
        final_grouped_orders.append(temp_group)

    return final_grouped_orders, grouped_stop_ids, group_weights, group_plts


시간 계산 함수

In [11]:
def min_cal(current_time, add_min = 0, plt = 0):
    # 현재 시간을 시와 분으로 분리
    current_hour = int(current_time)
    current_min = (current_time - current_hour)*100 

    add_time_hour = add_min //60
    add_time_min = add_min % 60 
    
    # plt의 개수당 10분을 추가
    additional_minutes = (plt * 10) % 60 
    additional_houres = (plt * 10) // 60

    # 총 분 계산 (기존 분 + 추가할 분 + plt에 의한 추가 분)
    new_time_hour = current_hour + add_time_hour + additional_houres
    new_time_min = current_min + add_time_min + additional_minutes
    new_time = new_time_hour + (new_time_min // 60) + (new_time_min % 60 /100)
        
    return new_time

이동시간, 이동거리 계산 함수

In [12]:
def moving_time(route, od_time):
    path = route.copy()
    path.insert(0,'Terminal')
    path.append('Terminal')
    moving_time_tmp = 0
    for i in range(len(path) - 1):
        moving_time_tmp += od_time.loc[path[i], path[i + 1]]
    out_time_tmp = od_time.loc[path[0], path[1]]  # 터미널에서 첫 착지로 이동하는데 걸린 시간
    in_time_tmp = od_time.loc[path[-2], path[-1]]  # 마지막 착지에서 터미널로 복귀하는데 걸린 시간
    working_time_tmp = moving_time_tmp - (out_time_tmp + in_time_tmp)  # 움직이는 총 시간
    moving_time = min_cal(0, moving_time_tmp, 0)
    working_time = min_cal(0, working_time_tmp, 0)
    out_time = min_cal(0, out_time_tmp, 0)
    in_time = min_cal(0, in_time_tmp, 0)
    return moving_time, working_time, out_time, in_time

def route_distance(route, od_distance):
    path = route.copy()
    path.insert(0,'Terminal')
    path.append('Terminal')
    distance = 0
    for i in range(len(path)-1):
        distance += od_distance.loc[path[i],path[i+1]]
    return distance

하차시간 계산 함수

In [13]:
def unloading_time(stop_id_list, plt_list): # plt에 따른 하차하는데 걸린 시간(분) (고정시간+변동시간)
    tmp = 0
    for i in range(len(stop_id_list)):
        tmp += 5
    for i in range(len(plt_list)):
        tmp += plt_list[i]*10
    unloading_time = tmp
    return unloading_time 

라우팅 알고리즘

In [14]:
# 초기해(근사 최적해)를 구하는 Nearest Neighbor 알고리즘
def nearest_neighbor(matrix):
    start_node = 'Terminal'
    n = len(matrix)
    visited = {node: False for node in matrix.index}
    path = [start_node]
    current_node = start_node
    visited[current_node] = True

    while len(path) < n:
        current_index = current_node
        next_node = None
        min_distance = float('inf')
        
        for node in matrix.index:
            if not visited[node] and matrix.loc[current_index, node] < min_distance:
                min_distance = matrix.loc[current_index, node]
                next_node = node
        
        if next_node is None:
            break
        
        path.append(next_node)
        visited[next_node] = True
        current_node = next_node
    path.append('Terminal')
    return path


# 초기해의 꼬인 경로를 푸는 알고리즘
def two_opt(path, matrix):
    def path_cost(path):
        cost = 0
        for i in range(len(path) - 1):
            cost += matrix.loc[path[i], path[i + 1]]
        return cost

    improved = True
    max_no_improve = 10  # 개선이 없을 때 최대 반복 횟수
    no_improve_count = 0

    while improved and no_improve_count < max_no_improve:
        improved = False
        for i in range(1, len(path) - 2):
            for j in range(i + 1, len(path)):
                if j - i == 1: 
                    continue
                new_path = path[:i] + path[i:j][::-1] + path[j:]
                if path_cost(new_path) < path_cost(path):
                    path = new_path
                    improved = True
                    no_improve_count = 0  # 개선이 있으면 카운터 초기화
        if not improved:
            no_improve_count += 1  # 개선이 없으면 카운터 증가
    return path

시간 제약에 따른 빈 분할

In [15]:
def Bin_Slicing(grouped_orders, grouped_stop_ids, group_weights, group_plts, time_windows): 
    grouped_stop_ids_list = grouped_stop_ids.copy()
    grouped_order_list = grouped_orders.copy()
    group_weights_list = group_weights.copy()
    group_plts_list = group_plts.copy()
    
    new_bin_list = []
    new_order_list = []
    new_weight_list = []
    new_plt_list = []
    
    new_moving_time_list = []
    new_working_time_list = []
    new_out_time_list = []
    new_in_time_list = []
    continue_check = True

    while grouped_stop_ids_list:  # 리스트에 요소가 남아있는 동안 반복
        
        if not grouped_order_list or not group_weights_list or not group_plts_list:
            break

        tmp_orders = grouped_order_list.pop(0)
        tmp_stop_ids = grouped_stop_ids_list.pop(0)  # 첫 번째 원소를 가져와서 제거
        tmp_weights = group_weights_list.pop(0)
        tmp_plts = group_plts_list.pop(0)

        for j in range(len(tmp_stop_ids)-1, -1, -1):
            tmp_orders_range = tmp_orders[:j+1]
            tmp_stop_ids_range = tmp_stop_ids[:j+1]
            tmp_weights_range = tmp_weights[:j+1]
            tmp_plts_range = tmp_plts[:j+1]

            od_distance, od_time = OD_Matrix(tmp_stop_ids_range)
            initial_path = nearest_neighbor(od_distance)
            optimized_path = two_opt(initial_path, od_distance)
            time_moving, time_working, time_out, time_in = moving_time(optimized_path, od_time) 
            unload_time = unloading_time(tmp_stop_ids_range, tmp_plts_range)
            total_time_working = min_cal(time_working, unload_time, 0)

            if total_time_working <= time_windows:
                
                new_order_list.append(tmp_orders[:j+1])
                new_bin_list.append(tmp_stop_ids[:j+1])
                new_weight_list.append(tmp_weights[:j+1])
                new_plt_list.append(tmp_plts[:j+1])

                distance, time = OD_Matrix(tmp_stop_ids[:j+1])
                initial = nearest_neighbor(distance)
                optimized = two_opt(initial, distance)
                new_time_moving, new_time_working, new_time_out, new_time_in = moving_time(optimized, time)
                new_unload_time = unloading_time(tmp_stop_ids_range, tmp_plts_range)
                new_total_time_working = min_cal(new_time_working, new_unload_time, 0)

                new_moving_time_list.append(new_time_moving)
                new_working_time_list.append(new_total_time_working)
                new_out_time_list.append(new_time_out)
                new_in_time_list.append(new_time_in)
                
                if j+1 < len(tmp_stop_ids):  # 남은 원소가 있는 경우만 추가
                    grouped_order_list.append(tmp_orders[j+1:])
                    grouped_stop_ids_list.append(tmp_stop_ids[j+1:])
                    group_weights_list.append(tmp_weights[j+1:])
                    group_plts_list.append(tmp_plts[j+1:])
                break
            
    # orders = []
    # for outer in new_order_list:
    #     temp_list = []
    #     for inner in outer:
    #         temp_list.extend(inner)
    #     orders.append(temp_list)
    if grouped_stop_ids == new_bin_list:
            continue_check = False

    return  new_bin_list, new_order_list, new_weight_list, new_plt_list, new_moving_time_list, new_working_time_list, new_out_time_list, new_in_time_list, continue_check

프로세스

In [16]:
def final_bin(cluster, num):
    # final_df = pd.DataFrame(columns=['Stop_list', 'weight', 'plt', 'moving_time', 'working_time', 'out_time', 'in_time', 'time_cluster', 'time_start', 'time_end'])
    final_df = pd.DataFrame(columns=['Stop_list', 'Order_list', 'weight', 'plt', 'moving_time', 'working_time', 'out_time', 'in_time', 'time_cluster', 'time_start', 'time_end'])
    row = []
    if num == 1:
        veh_ton = 1000
        veh_plt = 2.8
        veh_count = 25
    elif num == 3:
        veh_ton = 3500
        veh_plt = 8
        veh_count = 15
    else:
        veh_ton = 11000
        veh_plt = 16
        veh_count = 5
    
    for i in range(4):
        if i == 0:
            time_start = 5
            time_end = 10
            time_window = 5
        elif i == 1:
            time_start = 5
            time_end = 22
            time_window = 17
        elif i == 2:
            time_start = 14
            time_end = 17
            time_window = 3
        else:
            time_start = 9
            time_end = 11
            time_window = 2

        for j in range(len(cluster[i])):
            orders, stops, weights, plts = sort_and_make_bin(cluster[i][j], veh_ton, veh_plt, veh_count)
            bins, order, weight, plt, moving_time, working_time, out_time, in_time, bool = Bin_Slicing(orders, stops, weights, plts, time_window)
            
            for k in range(len(bins)):
                row.append({'Stop_list':bins[k], 'Order_list':order[k],'weight':weight[k], 'plt':plt[k], 'moving_time':moving_time[k], 'working_time':working_time[k], 'out_time':out_time[k], 'in_time':in_time[k], 'time_cluster':i, 'time_start':time_start, 'time_end':time_end})
    final_df = pd.DataFrame(row)
    
    return final_df

In [17]:
mon1_bin_df = final_bin(mon_cluster_1, 1)
mon3_bin_df = final_bin(mon_cluster_3, 3)
mon11_bin_df = final_bin(mon_cluster_11, 11)

tue1_bin_df = final_bin(tue_cluster_1, 1)
tue3_bin_df = final_bin(tue_cluster_3, 3)
tue11_bin_df = final_bin(tue_cluster_11, 11)

wed1_bin_df = final_bin(wed_cluster_1, 1)
wed3_bin_df = final_bin(wed_cluster_3, 3)
wed11_bin_df = final_bin(wed_cluster_11, 11)

thu1_bin_df = final_bin(thu_cluster_1, 1)
thu3_bin_df = final_bin(thu_cluster_3, 3)
thu11_bin_df = final_bin(thu_cluster_11, 11)

fri1_bin_df = final_bin(fri_cluster_1, 1)
fri3_bin_df = final_bin(fri_cluster_3, 3)
fri11_bin_df = final_bin(fri_cluster_11, 11)

sat1_bin_df = final_bin(sat_cluster_1, 1)
sat3_bin_df = final_bin(sat_cluster_3, 3)
sat11_bin_df = final_bin(sat_cluster_11, 11)

In [18]:
mon3_bin_df

,Stop_list,Order_list,weight,plt,moving_time,working_time,out_time,in_time,time_cluster,time_start,time_end
0,"[S_517, S_832]","[[O_302, O_1333, O_1334], [O_494, O_1173, O_11...","[3075.56, 422.8]","[5.11, 1.51]",3.006328,4.168328,0.0,0.0,0,5,10
1,"[S_904, S_465, S_459, S_491]","[[O_541, O_1661, O_1662, O_1663], [O_273, O_81...","[2276.44, 724.8199999999999, 396.76, 87.06]","[3.59, 1.11, 0.8, 0.33]",3.053908,4.236908,0.0,0.0,0,5,10
2,"[S_429, S_547]","[[O_255], [O_318, O_968, O_969]]","[1884.21, 1595.95]","[4.17, 2.98]",2.404127,4.019127,0.0,0.0,0,5,10
3,"[S_519, S_430, S_440]","[[O_304, O_861, O_862], [O_256, O_930, O_931],...","[1839.72, 1364.53, 264.0]","[2.56, 1.99, 0.44]",3.031228,4.080228,0.0,0.0,0,5,10
4,"[S_746, S_552, S_763]","[[O_444, O_1524, O_1525, O_1526, O_1527, O_152...","[1722.18, 1267.22, 231.7]","[2.83, 2.2600000000000002, 0.27]",3.169254,4.255254,0.0,0.0,0,5,10
...,...,...,...,...,...,...,...,...,...,...,...
255,[S_268],"[[O_160, O_1284, O_1285, O_1286]]",[1335.31],[2.01],1.347498,1.598498,0.0,0.0,3,9,11
256,[S_1055],"[[O_630, O_1287, O_1288]]",[1168.06],[1.27],0.237493,0.414493,0.0,0.0,3,9,11
257,[S_982],"[[O_584, O_1664, O_1665, O_1666, O_1667]]",[1519.3],[3.6900000000000004],0.313225,1.132225,0.0,0.0,3,9,11
258,[S_723],"[[O_430, O_766]]",[2943.2],[2.9600000000000004],1.193470,1.539470,0.0,0.0,3,9,11


Bin DataFrame의 weight와 plt를 합한 DataFrame 추출

In [19]:
def Reform_Bin(bin_df):
    sum_weight = []
    sum_plt = []
    for i in range(len(bin_df)):
        tmp_weight = sum(bin_df.loc[i, 'weight'])
        sum_weight.append(tmp_weight)
        tmp_plt = sum(bin_df.loc[i, 'plt'])
        sum_plt.append(tmp_plt)

    bin_df['weight'] = sum_weight
    bin_df['plt'] = sum_plt
    return bin_df

In [20]:
new_mon1_bin_df = Reform_Bin(mon1_bin_df[:])
new_mon3_bin_df = Reform_Bin(mon3_bin_df[:])
new_mon11_bin_df = Reform_Bin(mon11_bin_df[:])

new_tue1_bin_df = Reform_Bin(tue1_bin_df[:])
new_tue3_bin_df = Reform_Bin(tue3_bin_df[:])
new_tue11_bin_df = Reform_Bin(tue11_bin_df[:])

new_wed1_bin_df = Reform_Bin(wed1_bin_df[:])
new_wed3_bin_df = Reform_Bin(wed3_bin_df[:])
new_wed11_bin_df = Reform_Bin(wed11_bin_df[:])

new_thu1_bin_df = Reform_Bin(thu1_bin_df[:])
new_thu3_bin_df = Reform_Bin(thu3_bin_df[:])
new_thu11_bin_df = Reform_Bin(thu11_bin_df[:])

new_fri1_bin_df = Reform_Bin(fri1_bin_df[:])
new_fri3_bin_df = Reform_Bin(fri3_bin_df[:])
new_fri11_bin_df = Reform_Bin(fri11_bin_df[:])

new_sat1_bin_df = Reform_Bin(sat1_bin_df[:])
new_sat3_bin_df = Reform_Bin(sat3_bin_df[:])
new_sat11_bin_df = Reform_Bin(sat11_bin_df[:])

차량 배차 알고리즘

In [21]:
def Vehicle_Dicision(df):
    if df['weight'] <= 1000 and df['plt'] <= 2.8:
        return 'one'
    elif df['weight'] <= 2500 and df['plt'] <= 4:
        return 'two'
    elif df['weight'] <= 3500 and df['plt'] <= 8:
        return 'three'
    elif df['weight'] <= 5000 and df['plt'] <= 10:
        return 'five'
    elif df['weight'] <= 11000 and df['plt'] <= 16:
        return 'eleven'

In [22]:
# 차량 배차를 위한 임시 데이터프레임
tmp_veh1 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh2 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh3 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh5 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh11 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])

In [23]:
def Assignment_Vehicle(bin_df, date):
    global tmp_veh1, tmp_veh2, tmp_veh3, tmp_veh5, tmp_veh11
    for i in range(len(bin_df)):
        tmp_bin = bin_df.iloc[i]
        veh_type = Vehicle_Dicision(tmp_bin)
        
        if veh_type == 'one':
            tmp_veh = tmp_veh1
        elif veh_type == 'two':
            tmp_veh = tmp_veh2
        elif veh_type == 'three':
            tmp_veh = tmp_veh3
        elif veh_type == 'five':
            tmp_veh = tmp_veh5
        elif veh_type == 'eleven':
            tmp_veh = tmp_veh11

        if len(tmp_veh) == 0:
            out_time = tmp_bin['out_time']
            out_time = (out_time // 1) * 60 + (out_time % 1) * 100
            time_start = tmp_bin['time_start']
            time_start = min_cal(time_start, -out_time, 0)
            time_start = min_cal(time_start, -30, 0)
            moving_time = tmp_bin['moving_time']
            moving_time = (moving_time // 1) * 60 + (moving_time % 1) * 100
            if time_start < 5:
                time_start = 5
                moving_time += 30
            time_end = min_cal(time_start, moving_time, 0)
            new_row = pd.DataFrame({'first_route': [tmp_bin['Stop_list']], 'first_order': [tmp_bin['Order_list']], 'first_start': [time_start], 'first_end': [time_end], 'first_plt': [tmp_bin['plt']], 'first_weight': [tmp_bin['weight']]})
            tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)

        else:
            flag1 = False
            in_time = tmp_bin['in_time']
            in_time = (in_time // 1) * 60 + (in_time % 1) * 100
            time_end = tmp_bin['time_end']
            time_end = min_cal(time_end, in_time, 0)
            moving_time = tmp_bin['moving_time']
            moving_time = (moving_time // 1) * 60 + (moving_time % 1) * 100           
            if time_end > 24:
                time_end = 24
            time_start = min_cal(time_end, -moving_time, 0)
            time_start = min_cal(time_start, -30, 0)
            for j in range(len(tmp_veh)):
                if tmp_veh.iloc[j]['first_end'] < time_start and pd.isnull(tmp_veh.iloc[j]['second_start']):
                    tmp_veh.at[j, 'second_route'] = tmp_bin['Stop_list']
                    tmp_veh.at[j, 'second_order'] = tmp_bin['Order_list']
                    tmp_veh.at[j, 'second_start'] = time_start
                    tmp_veh.at[j, 'second_end'] = time_end   
                    tmp_veh.at[j, 'second_plt'] = tmp_bin['plt']
                    tmp_veh.at[j, 'second_weight'] = tmp_bin['weight']
                    flag1 = True
                    break

            if not flag1:
                flag2 = False
                out_time = tmp_bin['out_time']
                out_time = (out_time // 1) * 60 + (out_time % 1) * 100
                time_start = tmp_bin['time_start']
                time_start = min_cal(time_start, -out_time, 0)
                time_start = min_cal(time_start, -30, 0)
                
                if time_start < 5:
                    time_start = 5
                time_end = min_cal(time_start, moving_time, 0)

                for j in range(len(tmp_veh)):
                    if tmp_veh.iloc[j]['first_start'] > time_end and pd.isnull(tmp_veh.iloc[j]['second_start']):
                        tmp_veh.at[j, 'second_route'] = tmp_bin['Stop_list']
                        tmp_veh.at[j, 'second_order'] = tmp_bin['Order_list']
                        tmp_veh.at[j, 'second_start'] = time_start
                        tmp_veh.at[j, 'second_end'] = time_end
                        tmp_veh.at[j, 'second_plt'] = tmp_bin['plt']
                        tmp_veh.at[j, 'second_weight'] = tmp_bin['weight']
                        flag2 = True
                        break
                if not flag2:
                    new_row = pd.DataFrame({'first_route': [tmp_bin['Stop_list']], 'first_order': [tmp_bin['Order_list']], 'first_start': [time_start], 'first_end': [time_end], 'first_plt': [tmp_bin['plt']], 'first_weight': [tmp_bin['weight']]})
                    tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)

        # tmp_veh 데이터프레임을 업데이트된 상태로 유지
        if veh_type == 'one':
            tmp_veh1 = tmp_veh
        elif veh_type == 'two':
            tmp_veh2 = tmp_veh
        elif veh_type == 'three':
            tmp_veh3 = tmp_veh
        elif veh_type == 'five':
            tmp_veh5 = tmp_veh
        elif veh_type == 'eleven':
            tmp_veh11 = tmp_veh
    
    for i in [tmp_veh1, tmp_veh2, tmp_veh3, tmp_veh5, tmp_veh11]:
        date_row = [date]*len(i)
        i['date'] = date_row
    return tmp_veh1, tmp_veh2, tmp_veh3, tmp_veh5, tmp_veh11

In [24]:
mon_veh1, mon_veh2, mon_veh3, mon_veh5, mon_veh11 = Assignment_Vehicle(new_mon1_bin_df, 1)
mon_veh1, mon_veh2, mon_veh3, mon_veh5, mon_veh11 = Assignment_Vehicle(new_mon3_bin_df, 1)
mon_veh1, mon_veh2, mon_veh3, mon_veh5, mon_veh11 = Assignment_Vehicle(new_mon11_bin_df, 1)

C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecat

In [25]:
# 차량 배차를 위한 임시 데이터프레임
tmp_veh1 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh2 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh3 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh5 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh11 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])

In [26]:
tue_veh1, tue_veh2, tue_veh3, tue_veh5, tue_veh11 = Assignment_Vehicle(new_tue1_bin_df, 2)
tue_veh1, tue_veh2, tue_veh3, tue_veh5, tue_veh11 = Assignment_Vehicle(new_tue3_bin_df, 2)
tue_veh1, tue_veh2, tue_veh3, tue_veh5, tue_veh11 = Assignment_Vehicle(new_tue11_bin_df, 2)

C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecat

In [27]:
# 차량 배차를 위한 임시 데이터프레임
tmp_veh1 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh2 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh3 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh5 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh11 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])

In [28]:
wed_veh1, wed_veh2, wed_veh3, wed_veh5, wed_veh11 = Assignment_Vehicle(new_wed1_bin_df, 3)
wed_veh1, wed_veh2, wed_veh3, wed_veh5, wed_veh11 = Assignment_Vehicle(new_wed3_bin_df, 3)
wed_veh1, wed_veh2, wed_veh3, wed_veh5, wed_veh11 = Assignment_Vehicle(new_wed11_bin_df, 3)

C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecat

In [29]:
# 차량 배차를 위한 임시 데이터프레임
tmp_veh1 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh2 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh3 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh5 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh11 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])

In [30]:
thu_veh1, thu_veh2, thu_veh3, thu_veh5, thu_veh11 = Assignment_Vehicle(new_thu1_bin_df, 4)
thu_veh1, thu_veh2, thu_veh3, thu_veh5, thu_veh11 = Assignment_Vehicle(new_thu3_bin_df, 4)
thu_veh1, thu_veh2, thu_veh3, thu_veh5, thu_veh11 = Assignment_Vehicle(new_thu11_bin_df, 4)

C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecat

In [31]:
# 차량 배차를 위한 임시 데이터프레임
tmp_veh1 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh2 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh3 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh5 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh11 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])

In [32]:
fri_veh1, fri_veh2, fri_veh3, fri_veh5, fri_veh11 = Assignment_Vehicle(new_fri1_bin_df, 5)
fri_veh1, fri_veh2, fri_veh3, fri_veh5, fri_veh11 = Assignment_Vehicle(new_fri3_bin_df, 5)
fri_veh1, fri_veh2, fri_veh3, fri_veh5, fri_veh11 = Assignment_Vehicle(new_fri11_bin_df, 5)

C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecat

In [33]:
# 차량 배차를 위한 임시 데이터프레임
tmp_veh1 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh2 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh3 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh5 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])
tmp_veh11 = pd.DataFrame(columns=['VehID', 'date', 'first_route', 'first_order', 'first_start', 'first_end', 'first_plt', 'first_weight', 'second_route', 'second_order', 'second_start', 'second_end', 'second_plt', 'second_weight'])

In [34]:
sat_veh1, sat_veh2, sat_veh3, sat_veh5, sat_veh11 = Assignment_Vehicle(new_fri1_bin_df, 6)
sat_veh1, sat_veh2, sat_veh3, sat_veh5, sat_veh11 = Assignment_Vehicle(new_fri3_bin_df, 6)
sat_veh1, sat_veh2, sat_veh3, sat_veh5, sat_veh11 = Assignment_Vehicle(new_fri11_bin_df, 6)

C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tmp_veh = pd.concat([tmp_veh, new_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\180967973.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecat

In [35]:
veh1_dfs = [mon_veh1, tue_veh1, wed_veh1, thu_veh1, fri_veh1, sat_veh1]
veh2_dfs = [mon_veh2, tue_veh2, wed_veh2, thu_veh2, fri_veh2, sat_veh2]
veh3_dfs = [mon_veh3, tue_veh3, wed_veh3, thu_veh3, fri_veh3, sat_veh3]
veh5_dfs = [mon_veh5, tue_veh5, wed_veh5, thu_veh5, fri_veh5, sat_veh5]
veh11_dfs = [mon_veh11, tue_veh11, wed_veh11, thu_veh11, fri_veh11, sat_veh11]

veh1_df = pd.concat(veh1_dfs, ignore_index=True)
veh2_df = pd.concat(veh2_dfs, ignore_index=True)
veh3_df = pd.concat(veh3_dfs, ignore_index=True)
veh5_df = pd.concat(veh5_dfs, ignore_index=True)
veh11_df = pd.concat(veh11_dfs, ignore_index=True)

In [36]:
veh1_df

,VehID,date,first_route,first_order,first_start,first_end,first_plt,first_weight,second_route,second_order,second_start,second_end,second_plt,second_weight
0,NaN,1,[S_986],[[O_1354]],5,7.123136,1.00,990.36,[S_986],[[O_587]],7.476864,10.0,0.3,70.49
1,NaN,1,[S_575],"[[O_338, O_1115, O_1116, O_1117]]",5,5.499611,1.56,882.95,[S_372],"[[O_226, O_936, O_937, O_938]]",7.538073,10.0,2.39,723.62
2,NaN,1,"[S_614, S_844]","[[O_360, O_1148, O_1149, O_1150, O_1151], [O_5...",5,6.532411,2.20,937.31,[S_281],"[[O_171, O_1118]]",8.106178,10.0,1.8,947.97
3,NaN,1,"[S_871, S_676]","[[O_519, O_1473, O_1474, O_1475, O_1476, O_147...",5,8.492702,2.42,782.94,[S_658],[[O_1393]],19.488043,22.0,0.9,890.35
4,NaN,1,[S_625],"[[O_1945, O_1949, O_1942]]",5,7.501970,2.04,990.94,[S_658],"[[O_1392, O_1394, O_394]]",19.488043,22.0,1.81,799.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,NaN,6,"[S_858, S_718]","[[O_725, O_1854], [O_608]]",13.3,15.581816,0.78,547.12,NaN,NaN,NaN,NaN,NaN,NaN
452,NaN,6,[S_834],"[[O_2226, O_2227, O_2228]]",13.3,15.316432,2.07,951.55,NaN,NaN,NaN,NaN,NaN,NaN
453,NaN,6,"[S_830, S_579, S_777]","[[O_700], [O_490], [O_656, O_1180]]",13.3,15.486528,0.71,332.80,NaN,NaN,NaN,NaN,NaN,NaN
454,NaN,6,[S_124],[[O_101]],13.3,15.575044,1.33,705.72,NaN,NaN,NaN,NaN,NaN,NaN


차량 ID 추가하기

In [37]:
def Naming_ID(df, veh_kg):
    ID_row = []
    for i in range(len(df)):
        ID_row.append(f"Veh{veh_kg}_{i+1}")
    df['VehID'] = ID_row
    return df

In [38]:
veh1_df = Naming_ID(veh1_df, 1000)
veh2_df = Naming_ID(veh2_df, 2500)
veh3_df = Naming_ID(veh3_df, 3500)
veh4_df = Naming_ID(veh5_df, 5000)
veh5_df = Naming_ID(veh11_df, 11000)
veh1_df

,VehID,date,first_route,first_order,first_start,first_end,first_plt,first_weight,second_route,second_order,second_start,second_end,second_plt,second_weight
0,Veh1000_1,1,[S_986],[[O_1354]],5,7.123136,1.00,990.36,[S_986],[[O_587]],7.476864,10.0,0.3,70.49
1,Veh1000_2,1,[S_575],"[[O_338, O_1115, O_1116, O_1117]]",5,5.499611,1.56,882.95,[S_372],"[[O_226, O_936, O_937, O_938]]",7.538073,10.0,2.39,723.62
2,Veh1000_3,1,"[S_614, S_844]","[[O_360, O_1148, O_1149, O_1150, O_1151], [O_5...",5,6.532411,2.20,937.31,[S_281],"[[O_171, O_1118]]",8.106178,10.0,1.8,947.97
3,Veh1000_4,1,"[S_871, S_676]","[[O_519, O_1473, O_1474, O_1475, O_1476, O_147...",5,8.492702,2.42,782.94,[S_658],[[O_1393]],19.488043,22.0,0.9,890.35
4,Veh1000_5,1,[S_625],"[[O_1945, O_1949, O_1942]]",5,7.501970,2.04,990.94,[S_658],"[[O_1392, O_1394, O_394]]",19.488043,22.0,1.81,799.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451,Veh1000_452,6,"[S_858, S_718]","[[O_725, O_1854], [O_608]]",13.3,15.581816,0.78,547.12,NaN,NaN,NaN,NaN,NaN,NaN
452,Veh1000_453,6,[S_834],"[[O_2226, O_2227, O_2228]]",13.3,15.316432,2.07,951.55,NaN,NaN,NaN,NaN,NaN,NaN
453,Veh1000_454,6,"[S_830, S_579, S_777]","[[O_700], [O_490], [O_656, O_1180]]",13.3,15.486528,0.71,332.80,NaN,NaN,NaN,NaN,NaN,NaN
454,Veh1000_455,6,[S_124],[[O_101]],13.3,15.575044,1.33,705.72,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
vehicle1_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])
vehicle2_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])
vehicle3_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])
vehicle5_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])
vehicle11_info = pd.DataFrame(columns=['VehicleID', 'Round', 'OrderCount', 'StopCount', 'Plt', 'TravelDistance', 'WorkTime', 'TravelTime','UnloadingTime','WaitingTime','TotalCost','FixedCost','VariableCost'])

차량 결과 테이블 생성

In [40]:
def result_vehicle(veh_df, veh_info, fixed_cost, variable_cost):
    for i in range(len(veh_df)):
        tmp = veh_df.iloc[i]
        flag = pd.isnull(tmp['second_plt'])
        
        if flag:
            od_distance, od_time = OD_Matrix(tmp['first_route'])
            mt, wt, ot, it = moving_time(tmp['first_route'], od_time)
            working_time = (mt // 1) * 60 + (mt % 1) * 100
            working_time = working_time + 5 * len(tmp['first_route']) + 10 * tmp['first_plt']
            info_row = pd.DataFrame({
                'VehicleID': [tmp['VehID']],
                'Round': [1],
                'OrderCount': [len(tmp['first_order'])],
                'StopCount': [len(tmp['first_route'])],
                'Plt': [tmp['first_plt']],
                'TravelDistance': [route_distance(tmp['first_route'], od_distance)],
                'WorkTime': [working_time],
                'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                'UnloadingTime': [5 * len(tmp['first_route']) + 10 * tmp['first_plt']],
                'WaitingTime': [0],
                'TotalCost': [fixed_cost + (route_distance(tmp['first_route'], od_distance)) * variable_cost],
                'FixedCost': [fixed_cost],
                'VariableCost': [(route_distance(tmp['first_route'], od_distance)) * variable_cost]
            })
            veh_info = pd.concat([veh_info, info_row], ignore_index=True)

        else:
            if tmp['first_start'] < tmp['second_start']:
                # First route
                od_distance, od_time = OD_Matrix(tmp['first_route'])
                mt, wt, ot, it = moving_time(tmp['first_route'], od_time)
                working_time = (mt // 1) * 60 + (mt % 1) * 100
                working_time = working_time + 5 * len(tmp['first_route']) + 10 * tmp['first_plt']
                info_row = pd.DataFrame({
                    'VehicleID': [tmp['VehID']],
                    'Round': [1],
                    'OrderCount': [len(tmp['first_order'])],
                    'StopCount': [len(tmp['first_route'])],
                    'Plt': [tmp['first_plt']],
                    'TravelDistance': [route_distance(tmp['first_route'], od_distance)],
                    'WorkTime': [working_time],
                    'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                    'UnloadingTime': [5 * len(tmp['first_route']) + 10 * tmp['first_plt']],
                    'WaitingTime': [0],
                    'TotalCost': [fixed_cost + (route_distance(tmp['first_route'], od_distance)) * variable_cost],
                    'FixedCost': [fixed_cost],
                    'VariableCost': [(route_distance(tmp['first_route'], od_distance)) * variable_cost]
                })
                veh_info = pd.concat([veh_info, info_row], ignore_index=True)

                # Second route
                od_distance, od_time = OD_Matrix(tmp['second_route'])
                mt, wt, ot, it = moving_time(tmp['second_route'], od_time)
                working_time = (mt // 1) * 60 + (mt % 1) * 100
                working_time = working_time + 5 * len(tmp['second_route']) + 10 * tmp['second_plt']
                info_row = pd.DataFrame({
                    'VehicleID': [tmp['VehID']],
                    'Round': [2],
                    'OrderCount': [len(tmp['second_order'])],
                    'StopCount': [len(tmp['second_route'])],
                    'Plt': [tmp['second_plt']],
                    'TravelDistance': [route_distance(tmp['second_route'], od_distance)],
                    'WorkTime': [working_time],
                    'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                    'UnloadingTime': [5 * len(tmp['second_route']) + 10 * tmp['second_plt']],
                    'WaitingTime': [0],
                    'TotalCost': [fixed_cost + (route_distance(tmp['second_route'], od_distance)) * variable_cost],
                    'FixedCost': [fixed_cost],
                    'VariableCost': [(route_distance(tmp['second_route'], od_distance)) * variable_cost]
                })
                veh_info = pd.concat([veh_info, info_row], ignore_index=True)

            else:
                # Fisrt route
                od_distance, od_time = OD_Matrix(tmp['second_route'])
                mt, wt, ot, it = moving_time(tmp['second_route'], od_time)
                working_time = (mt // 1) * 60 + (mt % 1) * 100
                working_time = working_time + 5 * len(tmp['second_route']) + 10 * tmp['second_plt']
                info_row = pd.DataFrame({
                    'VehicleID': [tmp['VehID']],
                    'Round': [1],
                    'OrderCount': [len(tmp['second_order'])],
                    'StopCount': [len(tmp['second_route'])],
                    'Plt': [tmp['second_plt']],
                    'TravelDistance': [route_distance(tmp['second_route'], od_distance)],
                    'WorkTime': [working_time],
                    'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                    'UnloadingTime': [5 * len(tmp['second_route']) + 10 * tmp['second_plt']],
                    'WaitingTime': [0],
                    'TotalCost': [fixed_cost + (route_distance(tmp['second_route'], od_distance)) * variable_cost],
                    'FixedCost': [fixed_cost],
                    'VariableCost': [(route_distance(tmp['second_route'], od_distance)) * variable_cost]
                })
                veh_info = pd.concat([veh_info, info_row], ignore_index=True)

                # Second Route
                od_distance, od_time = OD_Matrix(tmp['first_route'])
                mt, wt, ot, it = moving_time(tmp['first_route'], od_time)
                working_time = (mt // 1) * 60 + (mt % 1) * 100
                working_time = working_time + 5 * len(tmp['first_route']) + 10 * tmp['first_plt']
                info_row = pd.DataFrame({
                    'VehicleID': [tmp['VehID']],
                    'Round': [2],
                    'OrderCount': [len(tmp['first_order'])],
                    'StopCount': [len(tmp['first_route'])],
                    'Plt': [tmp['first_plt']],
                    'TravelDistance': [route_distance(tmp['first_route'], od_distance)],
                    'WorkTime': [working_time],
                    'TravelTime': [(mt // 1) * 60 + (mt % 1) * 100],
                    'UnloadingTime': [5 * len(tmp['first_route']) + 10 * tmp['first_plt']],
                    'WaitingTime': [0],
                    'TotalCost': [fixed_cost + (route_distance(tmp['first_route'], od_distance)) * variable_cost],
                    'FixedCost': [fixed_cost],
                    'VariableCost': [(route_distance(tmp['first_route'], od_distance)) * variable_cost]
                })
                veh_info = pd.concat([veh_info, info_row], ignore_index=True)
    return veh_info


In [41]:
vehicle1_result = result_vehicle(veh1_df, vehicle1_info, 145400, 173)
vehicle2_result = result_vehicle(veh2_df, vehicle1_info, 173200, 173)
vehicle3_result = result_vehicle(veh3_df, vehicle1_info, 200000, 237)
vehicle5_result = result_vehicle(veh5_df, vehicle1_info, 234000, 355)
vehicle11_result = result_vehicle(veh11_df, vehicle1_info, 185200, 421)

C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\2636133458.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  veh_info = pd.concat([veh_info, info_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\2636133458.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  veh_info = pd.concat([veh_info, info_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\2636133458.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is

In [42]:
vehicle_result_dfs = [vehicle1_result, vehicle2_result, vehicle3_result, vehicle5_result, vehicle11_result]
vehicle_result = pd.concat(vehicle_result_dfs, ignore_index=True)

vehicle_result

,VehicleID,Round,OrderCount,StopCount,Plt,TravelDistance,WorkTime,TravelTime,UnloadingTime,WaitingTime,TotalCost,FixedCost,VariableCost
0,Veh1000_1,1,1,1,1.00,70.275041,117.313576,102.313576,15.0,0,157557.582166,145400,12157.582166
1,Veh1000_1,2,1,1,0.30,70.275041,110.313576,102.313576,8.0,0,157557.582166,145400,12157.582166
2,Veh1000_2,1,1,1,1.56,21.950981,70.561117,49.961117,20.6,0,149197.519778,145400,3797.519778
3,Veh1000_2,2,1,1,2.39,51.822795,125.092726,96.192726,28.9,0,154365.343565,145400,8965.343565
4,Veh1000_3,1,2,2,2.20,56.053911,145.241094,113.241094,32.0,0,155097.326663,145400,9697.326663
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2742,Veh11000_102,1,1,1,15.23,30.442602,206.827420,49.527420,157.3,0,198016.335565,185200,12816.335565
2743,Veh11000_102,2,2,2,14.67,199.226802,390.756824,234.056824,156.7,0,269074.483590,185200,83874.483590
2744,Veh11000_103,1,2,2,11.21,171.226664,336.854005,214.754005,122.1,0,257286.425340,185200,72086.425340
2745,Veh11000_103,2,1,1,9.13,28.133131,146.384417,50.084417,96.3,0,197044.047993,185200,11844.047993


In [43]:
# vehicle_result.to_csv('차량 결과 테이블.csv', index = False)

In [44]:
veh1_order_info = pd.DataFrame(columns=['Order_ID', 'VehicleID', 'Sequence', 'Stop_ID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DeaprtureTime'])
veh2_order_info = pd.DataFrame(columns=['Order_ID', 'VehicleID', 'Sequence', 'Stop_ID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DeaprtureTime'])
veh3_order_info = pd.DataFrame(columns=['Order_ID', 'VehicleID', 'Sequence', 'Stop_ID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DeaprtureTime'])
veh5_order_info = pd.DataFrame(columns=['Order_ID', 'VehicleID', 'Sequence', 'Stop_ID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DeaprtureTime'])
veh11_order_info = pd.DataFrame(columns=['Order_ID', 'VehicleID', 'Sequence', 'Stop_ID', 'ArrivalTime', 'WaitingTime', 'UnloadingTime', 'DeaprtureTime'])

In [45]:
# # def int_to_date(int_value, start_date='2024/09/'):
# #     date = '0' + str(int_value)
# #     ret_date = start_date + date
# #     return ret_date
# # kk = int_to_date(1)

# def date_time(int_value, str_value, start_date ='2024/09/'):
#     date = '0' + str(int_value)
#     ret_date = start_date + date

#     str_value = round(float(str_value), 2)
#     str_value = "{:.2f}".format(str_value)

#     time = str_value.replace('.', '/')
#     date_time = ret_date + "/" + time

#     return date_time



In [46]:
# def result_order(veh_df, order_info):
#     def calculate_unloading_time(order_id, day):
#         if day == 1:
#             df = order_mon_df
#         elif day == 2:
#             df = order_tue_df
#         elif day == 3:
#             df = order_wed_df
#         elif day == 4:
#             df = order_thu_df
#         elif day == 5:
#             df = order_fri_df
#         else:
#             df = order_sat_df
        
#         # 주문 ID가 데이터프레임에 있는지 확인
#         if order_id in df['Order_ID'].values:
#             return df[df['Order_ID'] == order_id]['plt'].values[0] * 10 + 5
#         else:
#             return 0  # 기본값을 0으로 설정 (또는 원하는 다른 값으로 설정)

#     def process_route(route, orders, start_time, sequence_offset=0):
#         nonlocal order_info  # nonlocal 키워드를 사용하여 외부 변수 참조
#         od_distance, od_time = OD_Matrix(route)
#         for j in range(len(route)):
#             moving_time = sum(od_time[route[m]][route[m+1]] for m in range(j))
#             arrival_time = min_cal(start_time, 30, 0)
#             arrival_time = min_cal(arrival_time, moving_time, 0)
#             for k in range(len(orders[j])):
#                 unloading_time = calculate_unloading_time(orders[j][k], tmp['date'])
#                 sequence = sequence_offset + j * len(orders[j]) + k + 1
#                 info_row = pd.DataFrame({
#                     'Order_ID': [orders[j][k]],
#                     'VehicleID': [tmp['VehID']],
#                     'Sequence': [sequence],
#                     'StopID': [route[j]],
#                     'ArrivalTime': [date_time(tmp['date'], [arrival_time])],
#                     'WaitingTime': [0],
#                     'UnloadingTime': [unloading_time],
#                     'DepartureTime': [date_time(tmp['date'],min_cal(arrival_time, unloading_time, 0))]
#                 })
#                 order_info = pd.concat([order_info, info_row], ignore_index=True)
#         return order_info

#     for i in range(len(veh_df)):
#         tmp = veh_df.iloc[i]
#         flag = pd.isnull(tmp['second_plt'])
        
#         if flag:
#             order_info = process_route(tmp['first_route'], tmp['first_order'], tmp['first_start'])
#         else:
#             if tmp['first_start'] < tmp['second_start']:
#                 order_info = process_route(tmp['first_route'], tmp['first_order'], tmp['first_start'])
#                 cnt = sum(len(o) for o in tmp['first_order'])
#                 order_info = process_route(tmp['second_route'], tmp['second_order'], tmp['second_start'], sequence_offset=cnt)
#             else:
#                 order_info = process_route(tmp['second_route'], tmp['second_order'], tmp['second_start'])
#                 cnt = sum(len(o) for o in tmp['second_order'])
#                 order_info = process_route(tmp['first_route'], tmp['first_order'], tmp['first_start'], sequence_offset=cnt)

#     return order_info


In [47]:
def date_time(int_value, str_value, start_date='2024/09/'):
    date = '0' + str(int_value)
    ret_date = start_date + date

    # str_value를 소수점 두 번째 자리까지 반올림하고 문자열로 변환
    str_value = round(float(str_value), 2)
    str_value = "{:.2f}".format(str_value)

    time = str_value.replace('.', '/')
    date_time = ret_date + "/" + time

    return date_time

def result_order(veh_df, order_info):
    def calculate_unloading_time(order_id, day):
        if day == 1:
            df = order_mon_df
        elif day == 2:
            df = order_tue_df
        elif day == 3:
            df = order_wed_df
        elif day == 4:
            df = order_thu_df
        elif day == 5:
            df = order_fri_df
        else:
            df = order_sat_df
        
        # 주문 ID가 데이터프레임에 있는지 확인
        if order_id in df['Order_ID'].values:
            return df[df['Order_ID'] == order_id]['plt'].values[0] * 10 + 5
        else:
            return 0  # 기본값을 0으로 설정 (또는 원하는 다른 값으로 설정)

    def process_route(route, orders, start_time, sequence_offset=0):
        nonlocal order_info  # nonlocal 키워드를 사용하여 외부 변수 참조
        od_distance, od_time = OD_Matrix(route)
        for j in range(len(route)):
            moving_time = sum(od_time[route[m]][route[m+1]] for m in range(j))
            arrival_time = min_cal(start_time, 30, 0)
            arrival_time = min_cal(arrival_time, moving_time, 0)
            for k in range(len(orders[j])):
                unloading_time = calculate_unloading_time(orders[j][k], tmp['date'])
                sequence = sequence_offset + j * len(orders[j]) + k + 1
                info_row = pd.DataFrame({
                    'Order_ID': [orders[j][k]],
                    'VehicleID': [tmp['VehID']],
                    'Sequence': [sequence],
                    'StopID': [route[j]],
                    'ArrivalTime': [date_time(tmp['date'], arrival_time)],
                    'WaitingTime': [0],
                    'UnloadingTime': [unloading_time],
                    'DepartureTime': [date_time(tmp['date'], min_cal(arrival_time, unloading_time, 0))]
                })
                order_info = pd.concat([order_info, info_row], ignore_index=True)
        return order_info

    for i in range(len(veh_df)):
        tmp = veh_df.iloc[i]
        flag = pd.isnull(tmp['second_plt'])
        
        if flag:
            order_info = process_route(tmp['first_route'], tmp['first_order'], tmp['first_start'])
        else:
            if tmp['first_start'] < tmp['second_start']:
                order_info = process_route(tmp['first_route'], tmp['first_order'], tmp['first_start'])
                cnt = sum(len(o) for o in tmp['first_order'])
                order_info = process_route(tmp['second_route'], tmp['second_order'], tmp['second_start'], sequence_offset=cnt)
            else:
                order_info = process_route(tmp['second_route'], tmp['second_order'], tmp['second_start'])
                cnt = sum(len(o) for o in tmp['second_order'])
                order_info = process_route(tmp['first_route'], tmp['first_order'], tmp['first_start'], sequence_offset=cnt)

    return order_info

# wlvlxl


In [48]:
order1_result = result_order(veh1_df, veh1_order_info)
order2_result = result_order(veh2_df, veh2_order_info)
order3_result = result_order(veh3_df, veh3_order_info)
order5_result = result_order(veh5_df, veh5_order_info)
order11_result = result_order(veh11_df, veh11_order_info)

C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\526353482.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  order_info = pd.concat([order_info, info_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\526353482.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  order_info = pd.concat([order_info, info_row], ignore_index=True)
C:\Users\정호준\AppData\Local\Temp\ipykernel_13824\526353482.py:55: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entri

In [49]:
order_result_dfs = [order1_result, order2_result, order3_result, order5_result, order11_result]
order_result = pd.concat(order_result_dfs, ignore_index=True)

order_result

,Order_ID,VehicleID,Sequence,Stop_ID,ArrivalTime,WaitingTime,UnloadingTime,DeaprtureTime,StopID,DepartureTime
0,O_1354,Veh1000_1,1,NaN,2024/09/01/5/30,0,15.0,NaN,S_986,2024/09/01/5/45
1,O_587,Veh1000_1,2,NaN,2024/09/01/8/18,0,8.0,NaN,S_986,2024/09/01/8/26
2,O_338,Veh1000_2,1,NaN,2024/09/01/5/30,0,10.7,NaN,S_575,2024/09/01/5/41
3,O_1115,Veh1000_2,2,NaN,2024/09/01/5/30,0,8.4,NaN,S_575,2024/09/01/5/38
4,O_1116,Veh1000_2,3,NaN,2024/09/01/5/30,0,7.1,NaN,S_575,2024/09/01/5/37
...,...,...,...,...,...,...,...,...,...,...
12357,O_1328,Veh11000_102,3,NaN,2024/09/06/19/28,0,0.0,NaN,S_712,2024/09/06/19/28
12358,O_2122,Veh11000_103,1,NaN,2024/09/06/5/30,0,0.0,NaN,S_1082,2024/09/06/5/30
12359,O_2328,Veh11000_103,2,NaN,2024/09/06/6/29,0,0.0,NaN,S_994,2024/09/06/6/29
12360,O_2046,Veh11000_103,3,NaN,2024/09/06/16/10,0,0.0,NaN,S_610,2024/09/06/16/10


In [50]:
import os

user_home = os.path.expanduser('~')
desktop_path = os.path.join(user_home, 'Desktop')


output_path = os.path.join(desktop_path, 'Order_Result.csv')


order_result.to_csv(output_path, index=False)